In [115]:
import nltk 
import warnings 
nltk.download('vader_lexicon')
warnings.filterwarnings('ignore')
import warnings 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/xuhuili/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [116]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint


In [117]:
date_sentiments = {}


In [123]:
for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/Tesla?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)


11 Oct 2019 https://www.businesstimes.com.sg/transport/tesla-cars-come-when-called-but-can-fray-nerves
05 Oct 2019 https://www.businesstimes.com.sg/transport/us-agency-reviews-whether-2000-tesla-electric-cars-should-have-been-recalled
03 Oct 2019 https://www.businesstimes.com.sg/transport/tesla-shares-fall-after-q3-deliveries-miss-estimates
03 Oct 2019 https://www.businesstimes.com.sg/transport/teslas-china-factory-said-to-be-on-schedule-to-start-production-in-october
30 Sep 2019 https://www.businesstimes.com.sg/transport/tesla-labour-practices-musk-tweet-broke-the-law-says-judge
28 Sep 2019 https://www.businesstimes.com.sg/transport/us-labour-judge-rules-that-tesla-broke-labour-law
25 Sep 2019 https://www.businesstimes.com.sg/transport/china%E2%80%99s-tesla-rival-nio-plunges-26-on-escalating-losses
24 Sep 2019 https://www.businesstimes.com.sg/transport/grab-faces-off-against-tesla-taxis-in-indonesias-cab-war
23 Sep 2019 https://www.businesstimes.com.sg/garage/grab-facing-off-against-t

In [127]:
date_sentiment = {}


In [128]:
for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())


In [129]:
print(date_sentiment)


{datetime.date(2019, 10, 20): 0.011, datetime.date(2019, 10, 19): -0.034, datetime.date(2019, 10, 12): 0.979, datetime.date(2019, 10, 6): -0.715, datetime.date(2019, 10, 4): 0.325, datetime.date(2019, 10, 1): -0.977, datetime.date(2019, 9, 29): -0.972, datetime.date(2019, 9, 26): -0.747, datetime.date(2019, 9, 25): 0.994, datetime.date(2019, 9, 24): 0.994, datetime.date(2019, 9, 22): 0.996, datetime.date(2019, 9, 14): -0.347, datetime.date(2019, 9, 3): 0.468, datetime.date(2019, 8, 31): 0.477, datetime.date(2019, 8, 24): 0.867, datetime.date(2019, 8, 21): 0.89, datetime.date(2019, 8, 20): 0.89, datetime.date(2019, 8, 18): -0.969, datetime.date(2019, 8, 12): -0.861, datetime.date(2019, 8, 10): -0.995, datetime.date(2019, 8, 8): -0.892, datetime.date(2019, 8, 1): 0.333, datetime.date(2019, 7, 30): 0.99, datetime.date(2019, 7, 27): -0.899, datetime.date(2019, 7, 26): 0.683, datetime.date(2019, 7, 17): -0.161, datetime.date(2019, 7, 4): 0.502, datetime.date(2019, 7, 3): 0.969, datetime.dat

In [130]:
# Backtesting

from __future__ import (absolute_import, division, print_function,
                            unicode_literals)

# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')


In [147]:
import warnings
warnings.filterwarnings('ignore')

import datetime
import os.path
import sys
import matplotlib
import matplotlib.pyplot as plt

import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
import backtrader.plot


In [148]:
matplotlib.use('Qt5Agg')
plt.switch_backend('Qt5Agg')


In [149]:
class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment


In [150]:
class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # sell if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.75:
                self.log('Sell CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # buy if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.75:
                self.log('Buy CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)


In [151]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Strategy
    cerebro.addstrategy(SentimentStrat)

    # Data Feed
    data = bt.feeds.YahooFinanceData(
        dataname = 'TSLA',
        fromdate = earliest_date,
        todate = datetime.datetime(2019,10,18),
        reverse = False
    )


In [152]:
cerebro.adddata(data)


In [153]:
cerebro.broker.setcash(100000.0)


In [154]:
cerebro.addsizer(bt.sizers.FixedSize, stake=5)

In [155]:
cerebro.broker.setcommission(commission=0.02)

In [156]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


In [157]:
cerebro.run()

2019-02-11, Close, 312.84
None
2019-02-12, Close, 311.81
None
2019-02-13, Close, 308.17
None
2019-02-14, Close, 303.77
None
2019-02-15, Close, 307.88
0.394
2019-02-19, Close, 305.64
0.982
2019-02-20, Close, 302.56
0.982
2019-02-21, Close, 291.23
0.982
2019-02-22, Close, 294.71
0.982
2019-02-25, Close, 298.77
0.982
2019-02-26, Close, 297.86
0.982
2019-02-27, Close, 314.74
-0.971
2019-02-28, Close, 319.88
0.686
2019-02-28, Sell CREATE, 319.88
2019-03-01, SELL EXECUTED, Price: 306.94, Cost: -1534.70, Comm 30.69
2019-03-01, Close, 294.79
0.686
2019-03-04, Close, 285.36
0.686
2019-03-05, Close, 276.54
0.686
2019-03-06, Close, 276.24
0.305
2019-03-07, Close, 276.59
0.964
2019-03-08, Close, 284.14
0.964
2019-03-11, Close, 290.92
0.964
2019-03-12, Close, 283.36
0.964
2019-03-13, Close, 288.96
-0.977
2019-03-13, Buy CREATE, 288.96
2019-03-14, BUY EXECUTED, Price: 292.45, Cost: -1534.70, Comm 29.25
2019-03-14, OPERATION PROFIT, GROSS 72.45, NET 12.51
2019-03-14, Close, 289.96
-0.977
2019-03-15, 

In [158]:
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 99879.94


In [159]:
cerebro.plot(iplot=False)

[[<Figure size 1280x960 with 5 Axes>]]

[[<Figure size 864x576 with 5 Axes>]]